In [81]:
library(tidyverse)
library(keras)
library(tensorflow)
library(reticulate)

In [82]:
install_tensorflow(extra_packages="pillow")


Installation complete.



In [83]:
install_keras()


Installation complete.



In [84]:
model_list <- load_model_tf("./dandelion model")
model <- model_list

In [77]:
change_brightness <- function(image, alpha, beta){
    new_image <- array(0, dim = dim(image))
    new_image <- pmax(0, pmin(alpha * image + beta, 225))
    return (new_image)
    }

In [78]:
CCP_Atk_Bright <- function(image, trans) {
    img <- image
    for(channel in 1:dim(img)[3]) {
        temp1 <- image[,,1]
        temp2 <- image[,,2]
        temp3 <- image[,,3]
        
        temp <- (temp1 * trans[channel,1]) + (temp2 * trans[channel, 2]) + (temp3 * trans[channel, 3])
        img[,,channel] <- temp/3
        }
     img1 <- change_brightness(img, 1, 30)
    return (img1)
    }
    
   

In [79]:
a <- runif(3, min = 0.0, max = 1.0)
b <- runif(3, min = 0.0, max = 1.0)
c <- runif(3, min = 0.0, max = 1.0)

In [94]:
#change model to be able to classify dandelion/grass

target_size <- c(224,224)
res <- data.frame(file = character(), class = character(), percent_dandelion = numeric(), percent_grass = numeric(), stringsAsFactors = FALSE)

f <- list.files("./grass")
for (i in f){
  test_image <- image_load(paste("./grass/",i,sep = ""),
                                  target_size = target_size)
  x <- image_to_array(test_image)
  x <- array_reshape(x, c(1, dim(x)))
  x <- x/255
  
  change_brightness()  
  CCP_Atk_Bright()  
    
  pred <- model %>% predict(x)
  if (pred[1,2] < 0.50){
    res <- rbind(res, data.frame(file = i, class = "not_grass", percent_dandelion = pred[1,1], percent_grass = 1- pred[1,1]))
  } else {
    res <- rbind(res, data.frame(file = i, class = "grass", percent_dandelion = pred[1,1], percent_grass = 1 - pred[1,1]))
  }
}

f <- list.files("./dandelions")
for (i in f){
  test_image <- image_load(paste("./dandelions/", i, sep=""),
                           target_size = target_size)
  x <- image_to_array(test_image)
  x <- array_reshape(x, c(1, dim(x)))
  x <- x/255
    
    change_brightness()
    CCP_Atk_Bright()  
    
    
  pred <- model %>% predict(x)
  if(pred[1,1]<0.50){
}
if (pred[1,1] < 0.50){
    res <- rbind(res, data.frame(file = i, class = "not_dandelion", percent_dandelion = pred[1,1], percent_grass = 1 - pred[1,1]))
} else {
    res <- rbind(res, data.frame(file = i, class = "dandelion", percent_dandelion = pred[1,1], percent_grass = 1 - pred[1,1]))
    }
}

print(res)



ERROR: Error in array(0, dim = dim(image)): argument "image" is missing, with no default
